# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv = pd.read_csv("/Users/anishachaudhari/data_science/Challenges/API challenge/python-api-challenge/output_files/weather_output.csv")
weather_csv


,Unnamed: 0,City,City ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bethel Park,5180199,20,US,1659469532,74,40.3276,-80.0395,82.51,4.61
1,1,Perth,2063523,75,AU,1659469761,75,-31.9333,115.8333,55.78,19.57
2,2,Kendari,1640344,96,ID,1659469761,94,-3.9450,122.4989,72.43,2.42
3,3,Anaconda,5637146,87,US,1659469761,21,46.1285,-112.9423,79.79,17.83
4,4,Bengkulu,1649150,39,ID,1659469714,92,-3.8004,102.2655,74.44,4.56
...,...,...,...,...,...,...,...,...,...,...,...
587,587,Naghadeh,122698,0,IR,1659469921,21,36.9553,45.3880,80.20,6.35
588,588,Zhigansk,2012530,82,RU,1659469921,69,66.7697,123.3711,45.46,16.40
589,589,Tezu,1254709,100,IN,1659469921,95,27.9167,96.1667,75.90,3.36
590,590,Boyolangu,1648082,100,ID,1659469922,96,-8.1181,111.8935,72.86,1.45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps

gmaps.configure(api_key=g_key)

# Use Lat and Lng as locations and Humidity as weight

locations = weather_csv[["Lat", "Lng"]]
weight = weather_csv["Humidity"]


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities to fit weather conditions


narrowed_df = weather_csv[(weather_csv["Max Temp"]<80) & 
                          (weather_csv["Max Temp"]>70) &
                          (weather_csv["Wind Speed"]<10) & 
                          (weather_csv["Cloudiness"]==0)]

narrowed_df.head()


,Unnamed: 0,City,City ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
35,35,Castro,3466704,0,BR,1659469735,49,-24.7911,-50.0119,71.47,1.25
109,109,Arraial do Cabo,3471451,0,BR,1659469789,83,-22.9661,-42.0278,75.16,9.22
263,263,Monforte de Lemos,3116478,0,ES,1659469833,59,42.5216,-7.5142,77.94,3.98
281,281,Arrondissement d'Ajaccio,3038333,0,FR,1659469589,69,42.0000,8.9167,77.05,2.30
289,289,Birjand,140463,0,IR,1659469840,25,32.8663,59.2211,75.67,6.91


In [6]:
weather_csv.loc[
      (weather_csv["Max Temp"]<80) 
     & (weather_csv["Max Temp"]>70) 
     & (weather_csv["Wind Speed"]<10)  
     & (weather_csv["Cloudiness"]==0),
    "City"
]


35                       Castro
109             Arraial do Cabo
263           Monforte de Lemos
281    Arrondissement d'Ajaccio
289                     Birjand
366                       Acarí
423                    A Coruña
454                      Grasse
464          La Baule-Escoublac
Name: City, dtype: object

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
35,Castro,BR,-24.7911,-50.0119,
109,Arraial do Cabo,BR,-22.9661,-42.0278,
263,Monforte de Lemos,ES,42.5216,-7.5142,
281,Arrondissement d'Ajaccio,FR,42.0000,8.9167,
289,Birjand,IR,32.8663,59.2211,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # no hotel found
    except (KeyError, IndexError):
        print("None Found")
              
hotel_df


Retrieving Results for Index 35: Castro.
Closest hotel in Castro is Hotel Buganville Express.
Retrieving Results for Index 109: Arraial do Cabo.
Closest hotel in Arraial do Cabo is Orlanova Boutique Hotel.
Retrieving Results for Index 263: Monforte de Lemos.
Closest hotel in Monforte de Lemos is Hotel Cardenal.
Retrieving Results for Index 281: Arrondissement d'Ajaccio.
Closest hotel in Arrondissement d'Ajaccio is Sole E Monte.
Retrieving Results for Index 289: Birjand.
Closest hotel in Birjand is مهمان پذیر نوید.
Retrieving Results for Index 366: Acarí.
Closest hotel in Acarí is HOTEL BELLA UNION.
Retrieving Results for Index 423: A Coruña.
Closest hotel in A Coruña is Hotel NH Collection A Coruña Finisterre.
Retrieving Results for Index 454: Grasse.
Closest hotel in Grasse is Auberge du Vieux Château.
Retrieving Results for Index 464: La Baule-Escoublac.
Closest hotel in La Baule-Escoublac is Hôtel Mercure La Baule Majestic.


,City,Country,Lat,Lng,Hotel Name
35,Castro,BR,-24.7911,-50.0119,Hotel Buganville Express
109,Arraial do Cabo,BR,-22.9661,-42.0278,Orlanova Boutique Hotel
263,Monforte de Lemos,ES,42.5216,-7.5142,Hotel Cardenal
281,Arrondissement d'Ajaccio,FR,42.0000,8.9167,Sole E Monte
289,Birjand,IR,32.8663,59.2211,مهمان پذیر نوید
366,Acarí,PE,-15.4311,-74.6158,HOTEL BELLA UNION
423,A Coruña,ES,43.3713,-8.3960,Hotel NH Collection A Coruña Finisterre
454,Grasse,FR,43.6667,6.9167,Auberge du Vieux Château
464,La Baule-Escoublac,FR,47.2922,-2.3640,Hôtel Mercure La Baule Majestic


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))